## Making Obsidian Graphs from Internet Browsing Sessions
This project takes your chrome history and turns it into an obsidian graph. The graph shows your browsing sessions, which are smartly summarized, categorized, and tagged using GPT and DBSCAN. 

To use this notebook, you'll need to set a filepath to your chrome history and provide your OpenAI API key. Running this notebook costs about 15c per 500 web browsing sessions.

Next Steps: 
- Better tagging & clustering systems
- Time-based filtering
- Bring in favorites/bookmarks?
- revisit page-based data
- more cleanup if people are interested in seeing this project develop (EG PCA is essentially unused in final output)

In [355]:
import pandas as pd
import sqlite3
from urllib.parse import urlparse, parse_qs
import re
from datetime import datetime
import numpy as np
import os
import ast
import openai
import plotly.express as px
from sklearn.decomposition import PCA
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [367]:
### Input these variables to get rollin'

# Set your OpenAI API key
openai.api_key = ''

# Chrome History File Location
history_filepath = 'History'
# Typically located at:
# Windows: C:\Users\<username>\AppData\Local\Google\Chrome\User Data\Default
# Mac: /Users/<username>/Library/Application Support/Google/Chrome/Default
# Linux: /home/<username>/.config/google-chrome/Default

# Obsidian vault filepath
obsidian_vault = 'obsidian_vault'

# Are you running this notebook for the first time, and generating summaries, embeddings, and tags?
# otherwise this notebook will try to load this data.
generate = True

# File name to save & later load your GPT generated browsing session descriptions
session_summaries = 'summaries.csv'

# File name to save & later load GPT generated embeddings
session_embeddings = 'embeddings.csv'

# File name for our assembled data
assembled_data = 'assembled_data.csv'

### More options
session_timeout = pd.Timedelta(minutes=30) # Define a session timeout threshold (e.g., 30 minutes)
# DBSCAN parameters, which determine our clustering
EPS = 40
MIN_SAMPLES = 2
FULL_EMBEDDINGS = True # True means DBSCAN will use all of our embedding dimensions for clustering, otherwise PCA

# Preparing Data

## Loading Data from your Chrome History file

In [10]:
conn = sqlite3.connect(history_filepath) # Opening Connection

# Reading Data
query = """
SELECT * 
FROM urls, visits
WHERE urls.id = visits.url
ORDER BY visits.visit_time DESC
"""

df = pd.read_sql_query(query, conn)

df.columns.values[8] = 'visit_id' # Rename the additional 'url' column
df['visit_time'] = pd.to_datetime((df['visit_time'] - 11644473600000000) // 1000000, unit='s') # create a readable timestamp
df = df.sort_values(by='visit_time')
conn.close() # Closing Connection

## Parsing URL into Components

In [11]:
def extract_subdomain(domain):
    parts = domain.split('.')
    if len(parts) > 2:
        return parts[0]
    return ''

def extract_file_extension(path):
    if '.' in path:
        return path.split('.')[-1]
    return ''

def extract_language_indicator(path):
    # Common language codes (extend this list as needed)
    lang_codes = ['en', 'es', 'fr', 'de', 'zh', 'ru', 'ja', 'pt', 'it']
    pattern = r'\/(' + '|'.join(lang_codes) + r')\/'
    match = re.search(pattern, path)
    return match.group(1) if match else ''

def parse_url(url):
    parsed_url = urlparse(url)
    domain = parsed_url.netloc
    path = parsed_url.path
    query_params = parse_qs(parsed_url.query)

    subdomain = extract_subdomain(domain)
    file_extension = extract_file_extension(path)
    language_indicator = extract_language_indicator(path)

    utms = {key: query_params[key][0] for key in query_params if key.startswith('utm_')}
    
    return domain, path, query_params, utms, subdomain, file_extension, language_indicator

# Apply the parsing function to the renamed URL column
df[['domain', 'path', 'query_params', 'utms', 'subdomain', 'file_extension', 'language_indicator']] = df['url'].apply(lambda x: pd.Series(parse_url(x)))

## Extracting Session Data

In [12]:
session_ids = [] # Initialize a list to store session ids
session_id = 0

# Iterate through the DataFrame and assign session ids
for i in range(1, len(df)):
    if df.iloc[i]['visit_time'] - df.iloc[i-1]['visit_time'] > session_timeout:
        session_id += 1
    session_ids.append(session_id)
    
# The first row is the start of the first session
session_ids.insert(0, 0)

# Add the session ids to the DataFrame
df['session_id'] = session_ids

df['session_id'].nunique() # peeking number of unique sessions

543

## Summarizing Sessions Based on Page Titles

In [122]:
# This function creates the text data we will pass to openai - it takes page titles in the session,
# limits them to 127 characters, and then joins them together
session_texts = df.groupby('session_id')['title'].apply(
    lambda titles: ' '.join(set(title[:127] + '|' for title in titles))
)
# session_texts # Uncomment to preview

In [164]:
# Creating our summarization function using GPT 3.5 Turbo
def summarize_text(text, model="gpt-3.5-turbo"):
    try:
        response = openai.ChatCompletion.create(
            model=model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant that comes up with succinct summaries for web browsing sessions. You don't say things like 'exploring' or other self-evident verbs."},
                {"role": "user", "content": "Come up with a concise title with no fluff or wasted wordcount that summarizes a browsing session with the following page titles: \n\n" + text}
            ],
            max_tokens=100,  # Adjust as needed
            temperature=0.7
        )
        summary = response['choices'][0]['message']['content'].strip()
        return summary
    except Exception as e:
        print(f"An error occurred: {e}")
        return None
    
# # Code for tuning the above Summarization function
# document = session_texts[100][:1024]
# summary = summarize_text(document)
# print("Summary:", summary)

In [162]:
### SPENDS MONEY ### Applying session summarization. Also takes time. Approximately 7 cents & 3 mins per 500 sessions
if generate == True:
    # Apply the summarize_text function to each text in session_texts
    summaries = [summarize_text(text[:1024]) for text in session_texts]
else:
    summaries = pd.read_csv(session_summaries)

In [170]:
# Converting our summaries to a dataframe
summaries_df = pd.DataFrame(summaries, columns=['Summaries'])

# Saving our summaries
if generate == True:
    summaries_df.to_csv(session_summaries, index=False)
    
# Creating a dictionary mapping the DataFrame's index to summaries
summary_dict = summaries_df['Summaries'].to_dict()

# Map our summaries to the DataFrame using the index
df['Summaries'] = df['session_id'].map(summary_dict)

### Data Cleaning - Identifying top domains, truncated versions of text fields, and visit time formats

In [353]:
# Fill any NaNs in title
df['title'].fillna('No Page Title', inplace=True)

# Ensure 'visit_time' is in datetime format
df['visit_time'] = pd.to_datetime(df['visit_time']) 
# Convert 'visit_time' to string for better display in hover data
df['visit_time_str'] = df['visit_time'].astype(str) 
# Convert 'visit_time' to a numerical format (e.g., seconds since the start of the dataset)
df['visit_time_numeric'] = (df['visit_time'] - df['visit_time'].min()).dt.total_seconds() 

# Find the top 15 most-visited domains (useful for visualizations)
top_domains = df['domain'].value_counts().nlargest(15).index 
# Add most-visited domains to a column in our dataframe
df['top_domain'] = df['domain'].apply(lambda x: x if x in top_domains else 'Other')

# Truncated versions of 'url', 'title', & 'summary' for labels
df['url_truncated'] = df['url'].str[:80]
df['title_truncated'] = df['title'].str[:80]
df['summary_truncated'] = df['Summaries'].str[:128]

## Generating Document Embeddings for Categorization & Mapping
Document Embeddings give us the ability to understand the semantic space of our earlier summarizations.

In [178]:
# Function for creating embeddings for semantic similarity of our document summaries
def generate_embeddings(text, model="text-embedding-3-small"):
    try:
        response = openai.Embedding.create(
            model=model,
            input=text
        )
        embedding = response['data'][0]['embedding']
        return embedding
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [188]:
# Get embeddings that describe our sessions - if generating, this takes time & some money (usually less than a penny)
if generate:
    # Generate embeddings from summaries and convert to DataFrame, then save them
    embeddings = [generate_embeddings(summary) for summary in summaries]
    embeddings_df = pd.DataFrame(embeddings)
    embeddings_df.to_csv(session_embeddings, index=False) # Save the DataFrame to CSV
else:
    # Load a saved embeddings DataFrame
    embeddings_df = pd.read_csv(session_embeddings)

# Creating an array from our saved embeddings DataFrame
embeddings_array = embeddings_df.to_numpy()
print(embeddings_array.shape)  # Display the shape of our embeddings array

(543, 1536)


## reduce our features to fewer dimensions using PCA

In [195]:
# Some due-diligence with embedding quality
# Filter out None values from embeddings
filtered_embeddings = [embedding for embedding in embeddings if embedding is not None]

# Find the maximum length among the non-None embeddings
max_length = max(len(embedding) for embedding in filtered_embeddings)

# Pad embeddings with zeros
padded_embeddings = [np.pad(embedding, (0, max_length - len(embedding)), 'constant') if embedding is not None else np.zeros(max_length) for embedding in embeddings]

# Convert the list of embeddings to a NumPy array
embeddings_array = np.array(padded_embeddings)

# Check if all embeddings are of the same length
if not all(len(embedding) == len(padded_embeddings[0]) for embedding in padded_embeddings):
    raise ValueError("Not all embeddings are of the same length.")

In [196]:
# Initialize PCA
pca = PCA(n_components=3)

# Fit and transform the data to reduce it to fewer dimensions
PCA_embeddings = pca.fit_transform(embeddings_array)

# Creating a df to hold our PCA results
PCA_df = pd.DataFrame(data=PCA_embeddings, columns=['PCA_X', 'PCA_Y', 'PCA_Z'])

### Creating a Session-based dataframe & adding our Principle Components

In [198]:
# Creating session_df based on 'session_id'
session_df = df.groupby('session_id').agg({
    'Summaries': 'first',
    'visit_time_str': lambda x: list(x),       # Collect all visit times in a list
    'title': lambda x: list(x),                # Collect all titles in a list
    'url': lambda x: list(x),                  # Collect all URLs in a list
    'domain': lambda x: list(x),                  # Collect all URLs in a list
    'visit_count': lambda x: list(x),          # Collect all visit counts in a list
    'typed_count': lambda x: list(x),          # Collect all typed counts in a list
    'top_domain': lambda x: list(x)      # Collect all domain categories in a list
}).reset_index()

# Merge PCA with session_df
PCA_df['session_id'] = sorted(pd.unique(session_df['session_id']))
session_df = session_df.merge(PCA_df, on='session_id', how='left')

# Final checks that no rows are being dropped
print("Final DataFrame size:", session_df.shape)
print("Session_id values:", len(session_df['session_id'].unique()))

Final DataFrame size: (543, 12)
Session_id values: 543


In [200]:
# Saving data
if generate == True: 
    session_df.to_csv(assembled_data, index=False)
else: # Import prepared data
    session_df = pd.read_csv(assembled_data)

# Now that data is prepared...

In [203]:
# session_df # Visually check our prepared data

# Check the shape to verify the structure
session_df.shape

(543, 12)

# Advanced Semantic Mapping
Next we'll use our embeddings to identify categories and themes in our sessions, and then export our categorized data into Obsidian.

### Appending embeddings to our session data

In [204]:
# Adding the embeddings array as a new column in the DataFrame
session_df['Document_Embeddings'] = list(embeddings_array)

In [205]:
# Verify the addition by checking the new DataFrame structure
len(session_df['Document_Embeddings'][0])
print(session_df.shape)
session_df.head()

1536

### Identifying clusters in embeddings using DBSCAN

In [328]:
# Uncomment to overwrite variables for tuning
# EPS = 40
# MIN_SAMPLES = 2
# FULL_EMBEDDINGS == True

# Create a StandardScaler object for Zscore Normalization
scaler = StandardScaler()

# Option to choose the type of embeddings: 'full = True' for Document_Embeddings, else just use PCA components
if FULL_EMBEDDINGS == True:
    # Assuming 'Document_Embeddings' is already in the correct format (list of arrays)
    DBSCAN_IN = np.array(session_df['Document_Embeddings'].tolist())
else:
    # Stack the PCA components into a single numpy array
    DBSCAN_IN = session_df[['Embedding_X', 'Embedding_Y', 'Embedding_Z']].values

# Normalize our input for DBSCAN
DBSCAN_IN = scaler.fit_transform(DBSCAN_IN)    

# Apply DBSCAN
# Note: Adjust eps and min_samples based on the scale and distribution of your data
dbscan = DBSCAN(eps=EPS, min_samples=MIN_SAMPLES)  # These parameters may need to be tuned
session_df['Cluster'] = dbscan.fit_predict(DBSCAN_IN)

# Filter out noise (-1 labels are considered noise in DBSCAN)
filtered_data = session_df[session_df['Cluster'] != -1]

In [330]:
# Code for tuning clustering parameters (EPS & MIN_SAMPLES)
# print(filtered_data.shape)
# display(filtered_data['Cluster'].value_counts())
# filtered_data # showing the rows that do have identified categories

(223, 14)


### Create descriptor tags for our clusters

In [336]:
def generate_tags(texts):
    # Ensure texts is a single string as the prompt expects a single coherent string input
    prompt = "Generate a list of 10 single-word tags that best describe the underlying conceptual categories the following descriptions all have in common: \n\n" + "\n".join(texts)
    
    # Using chat completions endpoint
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that operates as a concise category identifier."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=150,
        temperature=0.7,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    return response['choices'][0]['message']['content'].strip()

In [342]:
### COSTS MONEY ###
if generate == True:
    # Dictionary to hold descriptions for each cluster
    cluster_descriptions = {}
    
    # Generate descriptions for each cluster and store them in the dictionary
    for cluster_id in filtered_data['Cluster'].unique():
        cluster_summaries = filtered_data[filtered_data['Cluster'] == cluster_id]['Summaries'].unique()[:15]
        if len(cluster_summaries) > 0:
            description = generate_tags(cluster_summaries)
            cluster_descriptions[cluster_id] = description
        else:
            cluster_descriptions[cluster_id] = "No description available"
        
    # Map the descriptions back to the session_df based on Cluster IDs
    session_df['Cluster_Name'] = session_df['Cluster'].map(cluster_descriptions)

    # Write updated session_df to our file
    session_df.to_csv(assembled_data, index=False)    

else:
    session_df = pd.read_csv(assembled_data)

In [346]:
# Checking our results
# session_df[session_df['Cluster'] == 1].head(1)
# session_df['Cluster_Name'].value_counts()

### Creating an Obsidian Graph

### Some data cleaning

In [360]:
# Define a function to convert date strings from a specific pattern to 'mm/dd/yy' format
def convert_dates(date_str):
    date_str = str(date_str)  # Convert to string to ensure compatibility
    date_pattern = r"Timestamp\('(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2})'\)"
    dates = re.findall(date_pattern, date_str)
    
    if dates:
        first_date = datetime.strptime(dates[0], "%Y-%m-%d %H:%M:%S")
        return first_date.strftime("%m/%d/%y")
    return None

# Ensure there are no NaN values in the column to avoid type errors
session_df['visit_time_str'] = session_df['visit_time_str'].fillna('Unknown')

# Apply the conversion function to the 'visit_time_str' column
session_df['Simple_Date'] = session_df['visit_time_str'].apply(convert_dates)

In [381]:
# # Check the result
# print(session_df[['visit_time_str', 'Simple_Date']].head())

### Creating our Obsidian Vault

In [379]:
output_dir = obsidian_vault
os.makedirs(output_dir, exist_ok=True)

def sanitize_filename(text):
    sanitized = "".join([c if c.isalnum() or c in [' ', '-', '_'] else ' ' for c in text]).strip()
    return ' '.join(sanitized.split()[:20])[:150]

def extract_tags(cluster_name):
    tags = re.split(r'\d+\.\s*', cluster_name)
    return [tag.replace(' ', '_') for tag in tags if tag.strip()]

def create_markdown(row):
    filename = f"{sanitize_filename(row['Summaries'])}.md"
    filepath = os.path.join(output_dir, filename)
    
    # Normalize domain data, now just check if the list is empty or None
    domains = set(row['domain']) if row['domain'] else set()
    linked_domains = [f"[[{domain}]]" for domain in domains if domain.strip()]

    # Handling URLs which may already be a list or a string
    urls = row['url'] if isinstance(row['url'], list) else row['url'].split(',')
    
    with open(filepath, 'w', encoding='utf-8') as file:
        file.write("---\n")
        file.write(f"session_id: {row['session_id']}\n")
        file.write("tags:\n")
        tags = extract_tags(row['Cluster_Name']) if pd.notna(row['Cluster_Name']) else ["No_Category"]
        for tag in tags:
            file.write(f"  - {tag}\n")
        file.write("---\n\n")
        file.write(f"# {row['Summaries']}\n\n")
        file.write("**Visited Domains**:\n" + ' '.join(linked_domains) + "\n\n")
        file.write("<details>\n<summary>History Details</summary>\n")
        file.write("- **Full URLs**:\n" + '\n'.join([f"- {url}" for url in urls]) + "\n")
        file.write(f"- **Visited on**: {row['visit_time_str']}\n")
        file.write("</details>\n")

# Apply the function to each row in the DataFrame
session_df.apply(create_markdown, axis=1)

0      None
1      None
2      None
3      None
4      None
       ... 
538    None
539    None
540    None
541    None
542    None
Length: 543, dtype: object

# Summary

This was a fun project to learn more about Obsidian vaults & storage sytems, Chrome history file formats, DBSCAN, and working with OpenAI APIs to move up orders of abstraction to summarize and tag things. Navigating browser history & favorites folders is always a pain so it's fun to see some other possibilities, and while I think the result here is interesting I think in it's current state it's even less navigable & requires some more development (such as more time filtering, smarter tagging, and some other tricks) to become a useful tool.

I'm not sure if Obsidian is actually flexible enough a foundation for the features that this project may need, so I'll investigate what sort of plugin flexibility it has.

# Scrap Code

### Hashing Vectorizer
Hashing Vectorizer to PCA is fast & free but doesn't have semantic understanding & therefore has worse clustering.

In [219]:
# from sklearn.feature_extraction.text import HashingVectorizer

# # Define the number of features (dimensions) you want after hashing
# n_features = 6  # You can adjust this

# # apply hashing vectorizer to our session_summaries
# hasher = HashingVectorizer(n_features=n_features, alternate_sign=False)
# hashed_features = hasher.transform(df['session_summary'])

# # Convert to a dense array if it's not already
# hashed_features_dense = hashed_features.toarray()

# # Convert the dense array to a DataFrame
# hashed_df = pd.DataFrame(hashed_features_dense)

# # Optionally, prefix the column names to indicate they are hashed features
# hashed_df.columns = ['hash_feature_' + str(i) for i in range(hashed_df.shape[1])]

# # Reset index of your original DataFrame if necessary
# df.reset_index(drop=True, inplace=True)

# # Concatenate the DataFrames
# df_hash = pd.concat([df, hashed_df], axis=1)

In [366]:
# df_hash.head(2) # preview